In [6]:
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

import torchvision.transforms.functional as TF

DATA_DIR = "235746_카메라_품질_향상_AI경진대회_data/"

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root, isTrain, transform=None):
        self.root = root
        self.isTrain = isTrain
        self.transform = transform
        self.imgs_pd = pd.read_csv(self.root + "train.csv")
        self.imgs_path = self.root + "train_input_img/" + self.imgs_pd["input_img"].values
        self.labels_path = self.root + "train_label_img/" + self.imgs_pd["label_img"].values
        
    def __len__(self):
        return len(self.imgs_path)
    
    def __getitem__(self, idx):
        origin = np.array(Image.open(self.imgs_path[idx]).convert("RGB"))
        img = Image.open(self.imgs_path[idx]).convert("RGB")
        label = Image.open(self.labels_path[idx]).convert("RGB")
        
        if self.transform is not None:
            img = self.transform(img)
            label = self.transform(label)
            
        if self.isTrain:
            i, j, h, w = transforms.RandomCrop.get_params(
                img, output_size=(612, 816))
            img = TF.crop(img, i, j, h, w)
            label = TF.crop(label, i, j, h, w)
        
        return img, label

>**사진크기가 다른 경우가 있음**
>- transform 으로 사진 크기를 일정하게 맞추기
>- 작은 크기를 기준으로 사진이 크면 그 크기에 맞춰서 cropping??

In [9]:
train_transforms = transforms.Compose([transforms.ToTensor()])
valid_transforms = transforms.Compose([transforms.ToTensor()])
train_dataset = ImageDataset(DATA_DIR, True, train_transforms)
valid_dataset = ImageDataset(DATA_DIR, False, valid_transforms)

In [10]:
indices = torch.arange(0, len(train_dataset))
train_dataset = torch.utils.data.Subset(train_dataset, indices[: -100])
valid_dataset = torch.utils.data.Subset(valid_dataset, indices[-100: ])
len(train_dataset), len(valid_dataset)

(522, 100)

In [11]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False)

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b0')

In [ ]:
model

In [12]:
for img, label in valid_dataloader:
    print(img.size())
    pred = model(img)
    print(pred.size())
    break

torch.Size([1, 3, 1224, 1632])


NameError: name 'model' is not defined

In [13]:
from effunet import EffUNet

In [14]:
model = EffUNet(model="b4", 
                out_channels=1, 
               freeze_backbone=True,
               pretrained=True,
               device="cpu",
               )

Loaded pretrained weights for efficientnet-b4


In [19]:
import torch.nn.functional as F
import matplotlib.pyplot as plt

for img, label in train_dataloader:
    #print(ori.size())
    print(img.size())
    print(img.size() == (1, 3, 612, 816))
    tmp = F.interpolate(img, size=448)
    print(tmp.size())
    pred = model(tmp)
    print(pred.size())
    plt.imshow(ori[0])
    plt.show()
    plt.imshow(img[0].permute(1, 2, 0))
    plt.show()
    plt.imshow(label[0].permute(1, 2, 0))
    plt.show()
    plt.imshow(tmp[0].permute(1, 2, 0))
    plt.show()
    plt.imshow(pred[0].permute(1, 2, 0).detach().numpy())
    plt.show()
    break

torch.Size([1, 3, 612, 816])
True
torch.Size([1, 3, 448, 448])
torch.Size([1, 1, 448, 448])


NameError: name 'ori' is not defined

In [ ]:
model